In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
#import
import pandas as pd
import numpy as np
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE, Isomap
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, Normalizer
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import uniform, randint
import xgboost as xgb
from sklearn.metrics import auc, accuracy_score, mean_absolute_error
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, train_test_split, KFold
from sklearn.decomposition import PCA
from sklearn.multioutput import MultiOutputRegressor
import random

In [33]:
%cd /content/drive/MyDrive/Colab Notebooks/BigData_proj/model

df_old =  pd.read_csv('./input_formation.csv')

/content/drive/MyDrive/Colab Notebooks/BigData_proj/model


In [34]:
df = pd.read_csv('./input_formation.csv')
#all_list : tournament,home_team, away_team
all_list = ((df['tournament'].unique()).tolist()  +  (df['home_team'].unique()).tolist() + (df['away_team'].unique()).tolist())

uni_len = '***'.join(all_list)
nat_enc = tf.keras.preprocessing.text.Tokenizer(num_words=len(all_list),
    filters='',
    lower=False,
    split = '***')
nat_enc.fit_on_texts([uni_len])
nat_dic = nat_enc.word_index

In [35]:
#값을 enumerate로 변환
for step, val in enumerate(df['tournament']):
    df['tournament'].iloc[step] = (nat_dic.get(val))
for step, val in enumerate(df['home_team']):
    df['home_team'].iloc[step] = (nat_dic.get(val))
for step, val in enumerate(df['away_team']):
    df['away_team'].iloc[step] = (nat_dic.get(val))
for step, val in enumerate(df['neutral']):
    if df['neutral'].iloc[step] == False:
        df['neutral'].iloc[step] = 0
    if df['neutral'].iloc[step] == True:
        df['neutral'].iloc[step] = 1

In [38]:
#df_new
df_new = df
#len
temp_tor = []
for i in df_old['tournament']:
    temp_tor.append(len(i))
temp_away = []
for i in df_old['away_team']:
    temp_away.append(len(i))
temp_home = []
for i in df_old['home_team']:
    temp_home.append(len(i))
df_new['tor_len'] = temp_tor
df_new['home_len'] = temp_home
df_new['away_len'] = temp_away

# 결측값 있는 행 제거
df_new = df_new.dropna()

df_label1 = df_new['home_score']
df_label2 = df_new['away_score']

In [ ]:
#df_new, df_label 분리 : 모델의 인풋(df_new)과 아웃풋(df_label, home_score 과 away score)
df_new = df_new.drop(['home_score', 'away_score'], axis = 1)
#df_label
df_label = pd.concat([df_label1, df_label2],axis = 1)
df_label.columns = ['home_score', 'away_score']

In [40]:
x_train = df_new.to_numpy()
df_train = pd.DataFrame(data=x_train, columns=df_new.columns)

In [43]:
#x_train = df_new.to_numpy()
Norm = Normalizer()
x_temp = Norm.fit_transform(x_train)
df_temp = pd.DataFrame(data=x_temp, columns=df_new.columns)

In [44]:
df_corr = df_temp.corr()

In [ ]:
y_es = df_label.to_numpy()
x_es = df_temp.to_numpy()
x_t, x_val , y_t, y_val = train_test_split(x_es, y_es, random_state=28)

In [ ]:
params = {
    "estimator__colsample_bylevel": uniform(0.7, 0.3),
    "estimator__gamma": uniform(0, 0.5),
    "estimator__learning_rate": uniform(0.003, 0.3),
    "estimator__max_depth": randint(2, 6),
    "estimator__n_estimators": randint(100, 500), 
    "estimator__subsample": uniform(0.6, 0.4)
}

In [ ]:
#Multioutput regression are regression problems that involve predicting two or more numerical values given an input example.
xgb_model = MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror', eval_metric=['mae'], random_state = 1, use_label_encoder=False))

In [ ]:
#search : 하이퍼 파라미터 튜닝
search = RandomizedSearchCV(xgb_model, param_distributions=params, random_state =1, n_iter=5, cv=3,
                            verbose=0, n_jobs=1, return_train_score=True) 
search.fit(x_t, y_t) 

best_param = search.best_params_
print('-')
print(search.best_score_)
print(best_param)
print('-')

In [ ]:
#RandomizedSearchCV로 찾은 best param을 모델에 적용
xgb_model = MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror',eval_metric=['mae'], random_state = 1, 
                              use_label_encoder=False, **best_param)).fit(x_t, y_t) 

In [ ]:
#MAE : MSE와 유사한 loss function
#이 코드에서 loss function이 적용되는 부분이 어디지? 못 찾았음
print('MAE: ',mean_absolute_error(y_val, xgb_model.predict(x_val)))

In [ ]:
#K-fold 교차 검증:
#전체 훈련 데이터를 K개의 그룹으로 묶고, 그중 한 그룹을 검증셋으로 사용합니다. 
#이때 각각의 모든 데이터 그룹이 적어도 한 번씩은 검증셋으로 사용되어야 합니다. 
#K-fold 교차 검증을 사용함으로써 검정 데이터에 모델이 과적합 되는 것을 막을 수 있습니다. 
folds = KFold(n_splits = 5, shuffle = True, random_state = 100)
scores = cross_val_score(xgb_model, x_val, y_val, scoring='neg_mean_absolute_error', cv=folds, verbose=0)
print('KFold_Neg_MAE: ',np.mean(scores))

In [ ]:
#inp_list: 사용자가 입력한 home_team, away team
def predict_score(inp_list, autofill, Norm, seed, model):
    ind_list = []
    for i in inp_list:
        ind_list.append(nat_dic.get(i))
    seed[:,1] = ind_list[0]
    seed[:,2] = ind_list[1]
    x_test = Norm.fit_transform(seed)
    pred = model.predict(x_test)
    return np.ravel(pred).tolist()

In [ ]:
inp = str(input("Enter Home & Away team (like Home Away): "))
autofill = True
inp = inp.split(' ')

In [ ]:
#seed의 역할
seed = x_train[random.randint(0,x_train.shape[0])].reshape(1,-1)
seed = np.delete(seed, 4, axis=1)

In [ ]:
params = {
    'autofill': True,
    'Norm': Norm,
    'seed': seed,
    'model': xgb_model
}
ans = predict_score(inp, **params)